## Bibliotecas

In [98]:
!pip install mip

import pandas as pd
import numpy as np
import random
import math
import time
from mip import Model, xsum, minimize, BINARY, INTEGER, OptimizationStatus


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Mo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Leitura dos Dados

In [99]:
# Função para ler os dados do arquivo de instância no formato excel
# Parâmetros:
# - arquivo (str): Caminho para o arquivo Excel com os dados da instância do problema.
# Retorno: Retorna os parâmetros do problema (I, J, S, Z) e as matrizes de horários, recursos e custos.
def ler_dados(arquivo):
    dados = pd.read_excel(arquivo, sheet_name='inputgrad_certo', header=None)

    # Extrai os parâmetros
    I = int(dados.iloc[1, 0])  # Número de horários
    J = int(dados.iloc[1, 1])  # Número de turmas
    S = int(dados.iloc[4, 0])  # Número de salas
    Z = int(dados.iloc[4, 1])  # Número de recursos

    # Extrai as matrizes relacionadas aos horários, recursos e custos
    M_ini = 7
    M = dados.iloc[M_ini:M_ini + I, 0:J].values  # Matriz M (horários por turma)

    t_ini = M_ini + I + 2
    t = dados.iloc[t_ini:t_ini + S, 0:Z].values  # Matriz t (recursos por sala)

    r_ini = t_ini + S + 2
    r = dados.iloc[r_ini:r_ini + J, 0:Z].values  # Matriz r (recursos por turma)

    c_ini = r_ini + J + 2
    c = dados.iloc[c_ini:c_ini + S, 0:J].values  # Matriz c (custo de alocar uma turma em uma sala)

    w_ini = c_ini + S + 2
    w = dados.iloc[w_ini:w_ini + S, 0].values.reshape(-1, 1)  # Vetor w (capacidades das salas)

    q_ini = w_ini + S + 2
    q = dados.iloc[q_ini:q_ini + J, 0].values.reshape(-1, 1)  # Vetor q (número de alunos por turma)

    return I, J, S, Z, M, t, r, c.T, w, q

# Modelo

In [100]:
# Função para criar o modelo de alocação de aulas
# Parâmetros:
# - I, J, S, Z: Parâmetros extraídos dos dados (horários, turmas, salas e recursos).
# - M, t, r, c, w, q: Matrizes e vetores extraídos dos dados.
# Retorno: Retorna o modelo de otimização e as variaveis.
def criar_modelo(I, J, S, Z, M, t, r, c, w, q):
    modelo = Model("Alocacao_de_Salas", solver_name="CBC")

    x_ts = [[m.add_var(name=f'x_ts_{i}_{j}', var_type=BINARY) for j in range(s)] for i in range(t)]
    y_s = [m.add_var(name=f'y_{j}', var_type=INTEGER) for j in range(s)]

    alpha = 1
    beta = 1

    # Função Objetivo
    modelo.objective = minimize(
        alpha * sum(c[i][j] * x_ts[i][j] for i in range(t) for j in range(s)) +
        beta * sum(y_s[j] for j in range(s))
    )

    # Restrições
    # Restrição (2): Cada turma deve ser alocada em exatamente uma sala
    for i in range(t):
        m += sum(x_ts[i][j] for j in range(s)) == 1, f"alocar_turma_{i}"

    # Restrição (3): Turmas só podem ser alocadas em salas válidas
    for i in range(t):
        for j in range(s):
            m += x_ts[i][j] <= eta_ts[i][j], f"sala_valida_turma_{i}_sala_{j}"

    # Restrição (4): Conflito de horários
    for i in range(t):
        for j in range(t):
            if theta_att_prime[i][j] == 1:
                for k in range(s):
                    m += x_ts[i][k] + x_ts[j][k] <= 1, f"conflito_{i}_{j}_sala_{k}"

    # Restrição (5): Número de turmas alocadas em uma sala
    for j in range(s):
        m += sum(x_ts[i][j] for i in range(t)) <= y_s[j], f"turmas_na_sala_{j}"

    # Restrição (6): Variáveis binárias para alocação de turmas
    for i in range(t):
        for j in range(s):
            m += x_ts[i][j] >= 0, f"binaria_x_ts_{i}_{j}"

    # Restrição (7): Variáveis inteiras para número de turmas por sala
    for j in range(s):
        m += y_s[j] >= 0, f"inteira_ys_{j}"

    return modelo, x_ts, y_s

# Função para resolver o modelo de otimização com base em parâmetros fornecidos.
# Parâmetros:
# - modelo (pulp.LpProblem): O modelo de otimização que será resolvido.
# - presolve (int): Se 1, presolve é ativado; se 0, é desativado.
# - cortes (int): Intensidade da geracao de cortes (0 desativa).
# Retorno: O modelo resolvido.
def resolver_modelo(modelo, presolve, cortes):
    # Configurar as opções do solver
    modelo.preprocess = presolve  # Define preprocess (-1, 0, 1)
    modelo.cuts = cortes  # Define geração de cortes (-1, 0, 1, 2, 3)

    # Medir o tempo de execução
    start_time = time.time()

    # Resolver o modelo
    status = modelo.optimize(max_seconds=1800) # Limite de tempo de 30 minutos

    # Calcular o tempo de execução
    execution_time = time.time() - start_time

    # Exibir as informações do modelo resolvido
    print(f"Solução Encontrada: {modelo.objective_value}")
    print(f"Gap: {modelo.gap * 100:.2f}%")
    print(f"Tempo de execução: {execution_time:.2f} segundos")

    return modelo

# Função para exibir a solução do modelo de alocação
# Parâmetros:
# - modelo (mip.Model): O modelo de otimização resolvido.
# - x (list of lists): Variáveis de decisão de alocação turma-sala (binárias).
# - y (list): Variáveis de decisão de trocas de sala (inteiras).
# - J (int): Número de turmas.
# - S (int): Número de salas.
# Retorno: Nenhum. A função exibe diretamente a solução.
def exibir_solucao(modelo, x, y, J, S):
    # Verifica se o modelo encontrou uma solução
    if modelo.status == OptimizationStatus.OPTIMAL or modelo.status == OptimizationStatus.FEASIBLE:
        # Coletar solução de alocação de turmas
        alocacao = []
        for j in range(J):
            for s in range(S):
                if x[j][s].x >= 0.8:  # Verifica se a variável binária x[j][s] é 1 (alocada)
                    alocacao.append((f"Turma {j+1}", f"Sala {s+1}"))

        # Coletar o número de turmas por sala
        trocas = [y[s].x for s in range(S)]

        # Exibir a alocação de turmas e trocas de sala
        print("Alocação de turmas:")
        for turma, sala in alocacao:
            print(f"{turma} alocada em {sala}")

        print("\nTurmas por sala:")
        for s in range(S):
            print(f"Número de turmas alocadas na sala {s+1}: {trocas[s]}")
    else:
        print("Nenhuma solução viável foi encontrada.")

# Gerar Instâncias Maiores

In [101]:
# Função para gerar uma nova instância com base em dados lidos e um fator de escala.
# Esta função aumenta ou reduz o tamanho da instância conforme o fator de escala fornecido.
# Parâmetros:
# - I, J, S, Z: Parâmetros de horários, turmas, salas e recursos.
# - M, t, r, c, w, q, A_t: Matrizes lidas do arquivo de entrada (horários, recursos, custos, capacidade, alunos, etc).
# - escala: Fator de escala para aumentar ou reduzir o tamanho da instância (exemplo: 1.5 para aumentar em 50%).
# Retorno: Retorna os novos valores escalonados de I, J, S, Z e das matrizes M, t, r, c, w, q.
def gerar_instancia(I, J, S, Z, M, t, r, c, w, q, escala):
    novo_J = int(np.ceil(J * escala))
    novo_S = int(np.ceil(S * escala))
    novo_I = int(np.ceil(I * escala))
    novo_Z = int(np.ceil(Z * escala))

    # Função auxiliar para ampliar uma matriz de dados.
    def ampliar(matriz, novo_tamanho_linhas, novo_tamanho_colunas):
        fator_linhas = int(np.ceil(novo_tamanho_linhas / matriz.shape[0]))
        fator_colunas = int(np.ceil(novo_tamanho_colunas / matriz.shape[1]))

        matriz_expandida = np.tile(matriz, (fator_linhas, fator_colunas))

        # Corta a matriz expandida para o novo tamanho necessário.
        return matriz_expandida[:novo_tamanho_linhas, :novo_tamanho_colunas]

    # Ampliar as matrizes e vetores com base no novo tamanho.
    nova_M = ampliar(M, novo_I, novo_J)
    nova_t = ampliar(t, novo_S, novo_Z)
    nova_r = ampliar(r, novo_J, novo_Z)
    nova_c = ampliar(c, novo_J, novo_S)
    nova_w = np.tile(w, (int(np.ceil(novo_S / w.shape[0])), 1))[:novo_S]
    nova_q = np.tile(q, (int(np.ceil(novo_J / q.shape[0])), 1))[:novo_J]

    return novo_I, novo_J, novo_S, novo_Z, nova_M, nova_t, nova_r, nova_c, nova_w, nova_q

# Exemplo

In [ ]:
# OBS: Essa parte não funciona sem primeiro criar corretamente a função "criar_modelo"
escala = 1
I, J, S, Z, M, t, r, c, w, q = ler_dados('data.xlsx')

# Gerar a instância com base na escala fornecida
novo_I, novo_J, novo_S, novo_Z, nova_M, nova_t, nova_r, nova_c, nova_w, nova_q = gerar_instancia(I, J, S, Z, M, t, r, c, w, q, escala)

# Criar e resolver o modelo
mod, x, y = criar_modelo(novo_I, novo_J, novo_S, novo_Z, nova_M, nova_t, nova_r, nova_c, nova_w, nova_q)
m = resolver_modelo(mod, 1, 1)

# Exibir a solução (descomente a linha abaixo para exibir)
exibir_solucao(m, x, y, novo_J, novo_S)

FileNotFoundError: [Errno 2] No such file or directory: '../t1pm/data.xlsx'